In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

# Example: Custom dataset for time series
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = data
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        # Input: Sequence of length `seq_length`
        x = self.data[idx:idx + self.seq_length]
        # Target: The next value after the sequence
        y = self.data[idx + self.seq_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Generate dummy data
torch.manual_seed(42)
data = torch.sin(torch.linspace(0, 100, steps=500))  # Simulated chlorophyll data
seq_length = 12

# Create dataset and DataLoader
dataset = TimeSeriesDataset(data, seq_length)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: [Batch, Time steps, Features]
        output, (hidden, cell) = self.lstm(x)  # LSTM output
        hidden_out = hidden[-1]  # Take the last layer's hidden state
        return self.fc(hidden_out)

# Model parameters
input_size = 1  # Single feature (e.g., chlorophyll)
hidden_size = 64  # Number of LSTM units
num_layers = 2  # Number of LSTM layers
output_size = 1  # Predicting one value

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (x_batch, y_batch) in enumerate(dataloader):
        # Reshape input for LSTM: [Batch, Time steps, Features]
        x_batch = x_batch.unsqueeze(-1)  # Add the feature dimension
        y_batch = y_batch.unsqueeze(-1)  # Ensure target matches output shape
        
        # Forward pass
        predictions = model(x_batch)
        loss = criterion(predictions, y_batch)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

# Testing the model
model.eval()
with torch.no_grad():
    test_seq = data[-seq_length:].unsqueeze(0).unsqueeze(-1)  # Last sequence
    predicted = model(test_seq)
    print(f"Predicted value: {predicted.item():.4f}")



C:\Users\flore\AppData\Local\Temp\ipykernel_8708\1483180514.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


Epoch 1/10, Loss: 0.2293
Epoch 2/10, Loss: 0.0065
Epoch 3/10, Loss: 0.0013
Epoch 4/10, Loss: 0.0002
Epoch 5/10, Loss: 0.0002
Epoch 6/10, Loss: 0.0002
Epoch 7/10, Loss: 0.0001
Epoch 8/10, Loss: 0.0001
Epoch 9/10, Loss: 0.0000
Epoch 10/10, Loss: 0.0000
Predicted value: -0.3292
